<a href="https://colab.research.google.com/github/ijazahmad-star/Colab-Workspace/blob/main/llm_with_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-tavily langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.8 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-google-genai google-generativeai

INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent

In [ ]:
# !pip install -U langchain-anthropic

In [ ]:
from google.colab import userdata
t_api_key = userdata.get('TAVILY_API_KEY')
api_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
memory = MemorySaver()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key = api_key)
search = TavilySearch(max_results=2, tavily_api_key = t_api_key)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Ijaz and I live in Lahore.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Ijaz and I live in Lahore.
================================== Ai Message ==================================

Hello Ijaz from Lahore! How can I help you today?


In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

# input_message = {
#     "role": "user",
#     "content": "Hi, what is my name.",
# }
for step in agent_executor.stream(
    {"messages": "Hi, what is my name."}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, what is my name.
================================== Ai Message ==================================

Your name is Ijaz.


In [ ]:
# input_message = {
#     "role": "user",
#     "content": "What's the weather where I live?",
# }

# for step in agent_executor.stream(
#     {"messages": [input_message]}, config, stream_mode="values"
# ):
#     step["messages"][-1].pretty_print()

response = agent_executor.invoke({"input": "weather at my location?"}, config)

In [ ]:
response

{'messages': [HumanMessage(content="Hi, I'm Ijaz and I live in Lahore.", additional_kwargs={}, response_metadata={}, id='ac8f56c0-3f65-4b26-87e6-3643af1ab4d6'),
  AIMessage(content='Hello Ijaz from Lahore! How can I help you today? ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--70592d7d-30e8-4aad-a21d-2f54d63700fe-0', usage_metadata={'input_tokens': 1253, 'output_tokens': 13, 'total_tokens': 1266, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='Hi, what is my name.', additional_kwargs={}, response_metadata={}, id='cce433f7-23b4-45bc-854c-2f5ea1ae0ece'),
  AIMessage(content='Your name is Ijaz.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--4ae9953a-5d85-4d47-9a1a-0a65f21f65cb-0', usage_metadata={'input_tokens': 1276, 'output_tokens': 6, 'to

In [ ]:
input_message = {
    "role": "user",
    "content": "list down best hotels in lahore?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

list down best hotels in lahore?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (414c6546-bd10-4a28-8a3e-76844954b618)
 Call ID: 414c6546-bd10-4a28-8a3e-76844954b618
  Args:
    query: best hotels in Lahore
================================= Tool Message =================================
Name: tavily_search

{"query": "best hotels in Lahore", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.travelup.com/en-gb/blog/best-hotels-in-lahore", "title": "10 Best Hotels in Lahore to Stay in 2025", "content": "Lahore, a vibrant and historically rich city in Pakistan, boasts a diverse array of hotels catering to both local and international travellers. Furthermore, the city's central location ensures that hotels are conveniently situated near popular attractions, such as the Badshahi Mosque and Lahore Fort. Hotel

In [ ]:
input_message = {
    "role": "user",
    "content": "i am going to london suggest some places to visit?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

i am going to london suggest some places to visit?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (b7163511-6860-4d4c-a85a-53d304b28c8e)
 Call ID: b7163511-6860-4d4c-a85a-53d304b28c8e
  Args:
    query: places to visit in London
================================= Tool Message =================================
Name: tavily_search

{"query": "places to visit in London", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://destinationwellknown.com/things-to-do-london/", "title": "29 Best Things To Do in London + My Hidden Gems (2025)", "content": "* London **Book tickets for a bus tour in London here**. A boat trip on the Thames is, of course, one of the best ways to see many of London’s central sights, but if you want to get under the skin of the British capital and experience the city from one of the smaller can

In [ ]:
input_message = {
    "role": "user",
    "content": "Tell me available flight for dubai in oct 2025 from pakistan?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me available flight for dubai in oct 2025 from pakistan?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (7b755633-52f8-4461-bd34-f81dff208214)
 Call ID: 7b755633-52f8-4461-bd34-f81dff208214
  Args:
    query: flights from Pakistan to Dubai in October 2025
================================= Tool Message =================================
Name: tavily_search

{"query": "flights from Pakistan to Dubai in October 2025", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.wego.com/flights/pk/dxb/cheapest-flights-from-pakistan-to-dubai", "title": "Flight deals from: Pakistan to Dubai - Wego", "content": "Flight deals from: Pakistan to Dubai ; Islamabad · 144. 30 Oct - 22 Nov. 23 Days Round-trip ; Lahore · 183. 18 Oct - 05 Nov. 18 Days Round-trip ; Sialkot · 203. 21", "score": 0.98501, "raw_content": null}, {